## Ollama Embeddings

`OllamaEmbeddings`는 LangChain에서 Ollama 플랫폼의 다양한 임베딩 모델을 활용할 수 있게 해주는 통합 인터페이스입니다.


**특징**
- 로컬 실행: Ollama 서버를 통해 로컬에서 실행됩니다
- 빠른 속도: 로컬 서버를 통한 빠른 응답을 제공합니다
- 무료 사용: API 비용이 없습니다
- 개인정보 보호: 데이터가 외부로 전송되지 않습니다
- 다양한 모델: 여러 임베딩 모델을 지원합니다

**주요 매개변수**
- `model`: 사용할 Ollama 임베딩 모델명 (예: "nomic-embed-text", "snowflake-arctic-embed")
- `base_url`: Ollama 서버 URL (기본값: "http://localhost:11434") 
- `headers`: HTTP 헤더 설정 (선택사항)
- `num_thread`: 스레드 수 설정 (선택사항)

In [ ]:
# 임베딩 모델 다운로드 (처음 실행 시 필요)
# nomic-embed-text 모델은 다국어를 지원한다.
!ollama pull nomic-embed-text

In [ ]:
# Ollama 임베딩 모델 초기화
from langchain_ollama import OllamaEmbeddings

# 임베딩 모델 설정
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",  # Ollama 임베딩 모델 지정
    base_url="http://localhost:11434",  # Ollama 서버 URL (기본값)
)

print("Ollama 임베딩 모델 로딩 완료!")

In [ ]:
# 단일 쿼리 임베딩 생성
text = "LangChain은 대화형 AI 애플리케이션 개발을 위한 프레임워크입니다."
print(f"원본 텍스트: {text}")

# 임베딩 생성
query_embedding = embeddings.embed_query(text)

print(f"임베딩 차원: {len(query_embedding)}")
print(f"임베딩 벡터 (처음 5개 값): {query_embedding[:5]}")


In [ ]:
# 여러 문서의 배치 임베딩 생성
documents = [
    "LangChain은 대화형 AI 애플리케이션을 구축하기 위한 오픈소스 프레임워크입니다.",
    "Ollama는 로컬에서 대화형 모델을 실행할 수 있는 플랫폼입니다.",
    "임베딩은 텍스트를 고차원 벡터로 변환하는 기술입니다.",
    "벡터 데이터베이스는 임베딩을 효율적으로 저장하고 검색할 수 있습니다.",
    "RAG는 검색 증강 생성을 의미하며, 외부 지식을 활용하여 응답을 생성합니다."
]

print("문서 배치 임베딩 생성 중...")
doc_embeddings = embeddings.embed_documents(documents)

print(f"총 {len(doc_embeddings)}개 문서의 임베딩 생성 완료")

In [ ]:
# 임베딩 유사도 계산
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

print("\n=== 임베딩 유사도 계산 ===")

# 쿼리 임베딩 생성
query_text = "Ollama에 대해 알려주세요"
query_emb = embeddings.embed_query(query_text)

# 유사도 계산을 위한 벡터 변환
doc_matrix = np.array(doc_embeddings)
query_vector = np.array(query_emb).reshape(1, -1)

# 코사인 유사도 계산
similarities = cosine_similarity(query_vector, doc_matrix)[0]

print(f"쿼리: {query_text}")
print("\n문서별 유사도:")
for i, (sim, doc) in enumerate(zip(similarities, documents)):
    print(f"문서 {i+1} (유사도: {sim:.4f}): {doc[:50]}...")

# 가장 유사한 문서 출력
most_similar_idx = np.argmax(similarities)
print(f"\n가장 유사한 문서: {documents[most_similar_idx]}")
